In [8]:
"""
Tests for HDBSCAN clustering algorithm
Shamelessly based on (i.e. ripped off from) the DBSCAN test code
"""
import numpy as np
from scipy.spatial import distance
from scipy import sparse
from scipy import stats
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils._testing import (
    assert_array_equal,
    assert_array_almost_equal,
    assert_raises,
)
from hdbscan import (
    HDBSCAN,
    hdbscan,
    validity_index,
    approximate_predict,
    approximate_predict_scores,
    membership_vector,
    all_points_membership_vectors,
)

# from sklearn.cluster.tests.common import generate_clustered_data
from sklearn.datasets import make_blobs
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode

from tempfile import mkdtemp
from functools import wraps

from sklearn import datasets

import warnings

n_clusters = 3
# X = generate_clustered_data(n_clusters=n_clusters, n_samples_per_cluster=50)
X, y = make_blobs(n_samples=200, random_state=10)
X, y = shuffle(X, y, random_state=7)
X = StandardScaler().fit_transform(X)

X_missing_data = X.copy()
X_missing_data[0] = [np.nan, 1]
X_missing_data[5] = [np.nan, np.nan]



0.7397746616854031


In [ ]:
def test_hdbscan_distance_matrix():
    D = distance.squareform(distance.pdist(X))
    D /= np.max(D)

    labels, p, persist, ctree, ltree, mtree = hdbscan(D, metric="precomputed")
    # number of clusters, ignoring noise if present
    n_clusters_1 = len(set(labels)) - int(-1 in labels)  # ignore noise
    assert n_clusters_1 == n_clusters

    labels = HDBSCAN(metric="precomputed").fit(D).labels_
    n_clusters_2 = len(set(labels)) - int(-1 in labels)
    assert n_clusters_2 == n_clusters

    validity = validity_index(D, labels, metric="precomputed", d=2)
    print(validity)
    assert validity >= 0.6

test_hdbscan_distance_matrix()

In [10]:
def test_hdbscan_feature_vector():
    labels, p, persist, ctree, ltree, mtree = hdbscan(X)
    n_clusters_1 = len(set(labels)) - int(-1 in labels)
    assert n_clusters_1 == n_clusters

    labels = HDBSCAN().fit(X).labels_
    n_clusters_2 = len(set(labels)) - int(-1 in labels)
    assert n_clusters_2 == n_clusters

    validity = validity_index(X, labels)
    print(validity)
    assert validity >= 0.4

test_hdbscan_feature_vector()

0.7397746616854026


In [12]:
distance.squareform(distance.pdist(X))

array([[0.        , 0.3298585 , 2.58787844, ..., 3.13096099, 0.45522349,
        2.94719999],
       [0.3298585 , 0.        , 2.68891822, ..., 3.10298371, 0.57490871,
        2.99872532],
       [2.58787844, 2.68891822, 0.        , ..., 1.2263978 , 3.02582596,
        0.54685975],
       ...,
       [3.13096099, 3.10298371, 1.2263978 , ..., 0.        , 3.58460062,
        0.74819803],
       [0.45522349, 0.57490871, 3.02582596, ..., 3.58460062, 0.        ,
        3.39799078],
       [2.94719999, 2.99872532, 0.54685975, ..., 0.74819803, 3.39799078,
        0.        ]])

In [23]:
# distance.squareform()
D = distance.squareform(distance.pdist([[0,0],[1,1], [2,2]]))
D /= np.max(D)

In [24]:
D

array([[0. , 0.5, 1. ],
       [0.5, 0. , 0.5],
       [1. , 0.5, 0. ]])

In [28]:
distance.pdist([[0,0],[0,1], [1,1],[1,0]])

array([1.        , 1.41421356, 1.        , 1.        , 1.41421356,
       1.        ])